In [112]:
import pandas as pd
import numpy as np
import time

data = pd.read_csv('tiny.csv', header = None, names = ['x', 'y'])
cities = data[['x', 'y']].values  # assuming your CSV has 'x' and 'y' columns
n = len(cities)
data


,x,y
0,-1.085631,-0.678886
1,0.997345,-0.094709
2,0.282978,1.491390
3,-1.506295,-0.638902
4,-0.578600,-0.443982
5,1.651437,-0.434351
6,-2.426679,2.205930
7,-0.428913,2.186786
8,1.265936,1.004054
9,-0.866740,0.386186


In [114]:
# distance matrix
dist_matrix = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        dist_matrix[i, j] = np.linalg.norm(cities[i] - cities[j])


In [116]:
# brute force
from itertools import permutations


def compute_distance(tour_path):
    # given an order of cities to visit, and compute teh distance need to travel.
    dist = 0
    for i in range(len(tour_path) - 1):
        dist += dist_matrix[tour_path[i], tour_path[i+1]]
    # visit all the cities, but but not return to city1
    dist += dist_matrix[tour_path[i], tour_path[0]] # return to the starting point
    return dist

best_tour = None
min_distance = float('inf')


start_time = time.time()

for perm in permutations(range(1, n)):
    tour = (0,) + perm  # Prepend city 0 as the fixed starting point
    current_distance = compute_distance(tour)
    if current_distance < min_distance:
        min_distance = current_distance
        best_tour = tour
        
elapsed_time = time.time() - start_time

print("Using brute force: ")
print("time used: {:.4f} seconds".format(elapsed_time))
print("Best tour:", best_tour)
print("Minimum distance:", min_distance)

Using brute force: 
time used: 0.9588 seconds
Best tour: (0, 3, 4, 1, 5, 8, 2, 7, 9, 6)
Minimum distance: 12.640430188095191


the above code performs the brute force on 10cities, and find the best route starting from city[0]

# Integer Linear Programming

We introduce a martix X, for each pair of cities i and j. X_ij = 1 if the route from city i to j is included, and 0 otherwise
The goal is to minimize the total travel distance represented by "min \sum_i \sum_j d_{ij} X_{ij}"

Some constraints we have:
1. Each city is left exactly once: \sum_j X_{ij} = 1
2. Each city is entered exactly once: \sum_i X_{ij} = 1

In [104]:
import pulp

In [110]:

# Define the ILP problem
problem = pulp.LpProblem("TSP", pulp.LpMinimize)

# Decision variables: x[i][j] is 1 if the route from city i to j is used
x = pulp.LpVariable.dicts('x', (range(n), range(n)), cat='Binary')

# MTZ variables: u[i] helps to eliminate subtours (only needed for cities 1 to n-1)
u = pulp.LpVariable.dicts('u', range(n), lowBound=0, upBound=n, cat='Continuous')

# Objective function: minimize total travel distance
problem += pulp.lpSum(dist_matrix[i][j] * x[i][j] for i in range(n) for j in range(n)), "TotalDistance"

# Degree constraints: each city leaves and enters exactly once
for i in range(n):
    problem += pulp.lpSum(x[i][j] for j in range(n) if j != i) == 1, f"Out_{i}"
    problem += pulp.lpSum(x[j][i] for j in range(n) if j != i) == 1, f"In_{i}"

# MTZ Subtour elimination constraints (for cities 1 to n-1)
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            problem += u[i] - u[j] + n * x[i][j] <= n - 1, f"MTZ_{i}_{j}"

# Solve the problem using PuLP's default solver (CBC)
problem.solve()

# Output the results
# print("Status:", pulp.LpStatus[problem.status])
# print("Total distance:", pulp.value(problem.objective))
solution_edges = [(i, j) for i in range(n) for j in range(n) if pulp.value(x[i][j]) == 1]
print("Edges in the solution tour:", solution_edges)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/johnnyhuang/anaconda3/lib/python3.11/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/w8/6cb3vj053ps4rvg711krptd00000gn/T/b2d3243bd31a4990b69088e26892e503-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/w8/6cb3vj053ps4rvg711krptd00000gn/T/b2d3243bd31a4990b69088e26892e503-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 97 COLUMNS
At line 764 RHS
At line 857 BOUNDS
At line 957 ENDATA
Problem MODEL has 92 rows, 99 columns and 396 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10.5865 - 0.00 seconds
Cgl0004I processed model has 92 rows, 99 columns (90 integer (90 of which binary)) and 396 elements
Cbc0038I Initial state - 19 integers unsatisfied sum - 2
Cbc0038I Pass   1: suminf.    1.60000 (8) obj. 11.1255 iterations 24
Cbc0038I Pass   2: